In [17]:
import gensim

import pandas as pd


In [4]:
items_df = pd.read_csv(r"C:\Users\anjal\OneDrive\Documents\Deep-learning\20191226-items.csv")

# Read the reviews CSV
reviews_df = pd.read_csv(r"C:\Users\anjal\OneDrive\Documents\Deep-learning\20191226-reviews.csv")



In [5]:
print("Items columns:", items_df.columns.tolist())
print("Reviews columns:", reviews_df.columns.tolist())

Items columns: ['asin', 'brand', 'title', 'url', 'image', 'rating', 'reviewUrl', 'totalReviews', 'price', 'originalPrice']
Reviews columns: ['asin', 'name', 'rating', 'date', 'verified', 'title', 'body', 'helpfulVotes']


In [6]:
merged_df = pd.merge(reviews_df, items_df, on='asin', how='inner')

# Show merged result
print(merged_df.head())

         asin           name  rating_x               date  verified  \
0  B0000SX2UC          Janet         3   October 11, 2005     False   
1  B0000SX2UC     Luke Wyatt         1    January 7, 2004     False   
2  B0000SX2UC         Brooke         5  December 30, 2003     False   
3  B0000SX2UC  amy m. teague         3     March 18, 2004     False   
4  B0000SX2UC  tristazbimmer         4    August 28, 2005     False   

                                        title_x  \
0                   Def not best, but not worst   
1                   Text Messaging Doesn't Work   
2                               Love This Phone   
3                       Love the Phone, BUT...!   
4  Great phone service and options, lousy case!   

                                                body  helpfulVotes brand  \
0  I had the Samsung A600 for awhile which is abs...           1.0   NaN   
1  Due to a software issue between Nokia and Spri...          17.0   NaN   
2  This is a great, reliable phone. I 

In [7]:
merged_df.shape

(67986, 17)

In [9]:
merged_df.title_x[0]

'Def not best, but not worst'

In [18]:
from gensim.utils import simple_preprocess
import pandas as pd

def safe_preprocess(x):
    if isinstance(x, str):
        return simple_preprocess(x)
    else:
        return []

# Apply safely to title_x
title_tokens = merged_df['title_x'].apply(safe_preprocess)


In [19]:
merged_df['title_tokens'] = title_tokens
print(merged_df[['title_x', 'title_tokens']].head())


                                        title_x  \
0                   Def not best, but not worst   
1                   Text Messaging Doesn't Work   
2                               Love This Phone   
3                       Love the Phone, BUT...!   
4  Great phone service and options, lousy case!   

                                        title_tokens  
0                  [def, not, best, but, not, worst]  
1                     [text, messaging, doesn, work]  
2                                [love, this, phone]  
3                            [love, the, phone, but]  
4  [great, phone, service, and, options, lousy, c...  


In [20]:
from gensim.utils import simple_preprocess

review_text = merged_df['title_x'].apply(lambda x: simple_preprocess(x) if isinstance(x, str) else [])


In [21]:
review_text.loc[0]

['def', 'not', 'best', 'but', 'not', 'worst']

In [25]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [26]:
model.build_vocab(review_text, progress_per=1000)

In [27]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(963277, 1432780)

In [28]:

model.save("./word2vec-amazon-cell-accessories-reviews-short.model")

In [29]:
model.wv.most_similar("bad")

[('terrible', 0.8754389882087708),
 ('horrible', 0.8390907049179077),
 ('long', 0.7683793306350708),
 ('okay', 0.7662454843521118),
 ('poor', 0.7608432769775391),
 ('sucks', 0.7503284811973572),
 ('good', 0.7500685453414917),
 ('awful', 0.7446735501289368),
 ('nice', 0.7357691526412964),
 ('ok', 0.7332719564437866)]

In [30]:
model.wv.similarity(w1="cheap", w2="inexpensive")

0.88993746

In [31]:
model.wv.similarity(w1="great", w2="good")

0.912018